# 이미지 변환

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from pyts.image import GramianAngularField, MarkovTransitionField

## 1. 파일 불러오기

In [13]:
window_size = 5
overlap = 0
sfreq = 256
image_size = 1
data_dir = '05_70hz'

In [30]:
gaf_save_dir = f'{data_dir}/GAF/19ch_GAF_{window_size}s_{overlap}s_ol'
mtf_save_dir = f'{data_dir}/MTF/19ch_MTF_{window_size}s_{overlap}s_ol'
scalo_save_dir = f'{data_dir}_100uV_reject2/scalogram/19ch_scalogram_{window_size}s_{overlap}s_ol'

In [31]:
filepath = f'../../data/{data_dir}'

filenames = glob(f'{filepath}/*.csv')
filenames.sort()
filenames[:5]

['../../data/05_70hz/H S1 EC.csv',
 '../../data/05_70hz/H S10 EC.csv',
 '../../data/05_70hz/H S11 EC.csv',
 '../../data/05_70hz/H S13 EC.csv',
 '../../data/05_70hz/H S14 EC.csv']

In [32]:
raw_df_dict = {}
for file in filenames:
    df = pd.read_csv(file, index_col=0)
    file_modify = file.split('/')[-1][:-4] # 파일 이름만 추출
    raw_df_dict[file_modify] = df.drop(columns=['time', 'A2'])

### segmentation 함수

In [33]:
# 슬라이딩 윈도우 함수 정의
# 정확하게 떨어지지 않아 끝에 남는 부분은 버려짐
def sliding_window(df, window_size, overlap):
    step = window_size - overlap
    windows = [df.iloc[i:i + window_size] for i in range(0, len(df) - window_size + 1, step)]
    return windows

## GAF 저장

### epoch로 나누어진 데이터 저장

In [19]:
## GAF 변환 (epoched data)

import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField
from concurrent.futures import ProcessPoolExecutor
import torch
from torchvision import transforms

def process_file(file_name, trans_size=1, resized_size=(224, 224)):
    subj_name = file_name.split('/')[-1][:-4]
    data = pd.read_csv(file_name, index_col=0).drop(columns=['time', 'condition', 'A2'])
    epoch_idx = data['epoch'].unique()
    
    for idx in epoch_idx:
        window = data[data['epoch'] == idx].drop(columns=['epoch']).T
        if trans_size == 1:
            gaf = GramianAngularField(method='summation', overlapping=False)
        else:
            gaf = GramianAngularField(image_size=trans_size, method='summation', overlapping=False)

        trans_data = gaf.fit_transform(window.sort_index()).astype(np.float32)

        # min-max 정규화 (gray-scale 이미지 생성을 위해)
        for j in range(trans_data.shape[0]):
            min_val, max_val = np.min(trans_data[j, :, :]), np.max(trans_data[j, :, :])
            trans_data[j, :, :] = (trans_data[j, :, :] - min_val) / (max_val - min_val)

        resized_img = transforms.Resize(resized_size)(torch.from_numpy(trans_data)) # (c, h, w) 바로 resize
        resized_img = resized_img.numpy()
        
        if np.isnan(resized_img).any():
            print('NaN detected')
            continue

        np.save(f'../../images2/{gaf_save_dir}_fold/{subj_name}_{idx:03d}th_window.npy', resized_img)

    return f'{subj_name} 처리 완료'

def main():
    os.makedirs(f'../../images2/{gaf_save_dir}_fold', exist_ok=True)
    
    # 사용할 프로세스 수 설정 (CPU 코어 수에 따라 조정 가능)
    num_processes = 15

    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        futures = [executor.submit(process_file, file_name) for file_name in filenames]
        
        for future in tqdm(futures, total=len(filenames), desc="전체 진행률"):
            print(future.result())

if __name__ == '__main__':
    import matplotlib
    matplotlib.use('Agg')
    main()

전체 진행률:   1%|          | 1/119 [00:50<1:39:33, 50.63s/it]

H S1 EC 처리 완료


전체 진행률:   2%|▏         | 2/119 [01:04<56:43, 29.09s/it]  

H S1 EO 처리 완료


전체 진행률:   3%|▎         | 3/119 [01:13<38:19, 19.82s/it]

...

MDD S5 EC 처리 완료


전체 진행률:  98%|█████████▊| 117/119 [07:19<00:03,  1.88s/it]

MDD S5 EO 처리 완료
MDD S6 EC 처리 완료
MDD S6 EO 처리 완료
MDD S7 EC 처리 완료
MDD S9 EC 처리 완료


전체 진행률: 100%|██████████| 119/119 [07:23<00:00,  3.72s/it]

MDD S9 EO 처리 완료


### epochs로 나누어 지지 않은 데이터 저장

In [25]:
# GAF 변환 (not epoched)

import os
import numpy as np
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from pyts.image import GramianAngularField
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms


def process_window(k, i, window, trans_size=1, resized_size=(224, 224)):
    if trans_size == 1:
        gaf = GramianAngularField(overlapping=False) 
    else:
        gaf = GramianAngularField(image_size=trans_size, overlapping=False) # GAF using PAA for time series reduction

    trans_data = gaf.fit_transform(window.T.sort_index()).astype(np.float32)
    
    # Min-max normalization (to create a grayscale image)
    for j in range(trans_data.shape[0]):
        min_val, max_val = np.min(trans_data[j, :, :]), np.max(trans_data[j, :, :])
        trans_data[j, :, :] = (trans_data[j, :, :] - min_val) / (max_val - min_val)

    resized_img = transforms.Resize(resized_size)(torch.from_numpy(trans_data)) # (c, h, w) 바로 resize
    resized_img = resized_img.numpy()
    
    if np.isnan(resized_img).any():
        print('NaN detected', k, {i + 1})
        return

    # Save the normalized data
    np.save(f'../../images2/{gaf_save_dir}_fold/{k}_{i + 1:03d}th_window.npy', resized_img)

    return f'{k} {i+1}th window 처리 완료'


def main():
    fold_window_dict = {}

    for k in raw_df_dict:
        fold_window_dict[k] = sliding_window(raw_df_dict[k], window_size * sfreq, overlap * sfreq)

    os.makedirs(f'../../images2/{gaf_save_dir}_fold', exist_ok=True)

    tasks = []
    num_processes = 15

    # Create a ProcessPoolExecutor to manage the pool of processes
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        for k in tqdm(fold_window_dict):
            for i, window in enumerate(fold_window_dict[k]):
                # Submit the processing function to the pool
                tasks.append(executor.submit(process_window, k, i, window, trans_size=1, resized_size=(224, 224)))
        
        # Wait for all tasks to complete
        for task in tqdm(tasks):
            task.result()


if __name__ == '__main__':
    import matplotlib
    matplotlib.use('Agg')
    main()

100%|██████████| 8860/8860 [07:53<00:00, 18.70it/s]


### 한 폴더에 저장(채널 선택)

In [ ]:
# # k-fold를 위해 한번에 저장 및 채널 선택해 저장
# fold_window_dict = {}
# channels = ['P3', 'P4']

# for k in raw_df_dict:
#     fold_window_dict[k] = sliding_window(raw_df_dict[k], window_size * sfreq, overlap * sfreq)

# os.makedirs(f'../../images2/{gaf_save_dir}_fold_{"_".join(channels)}', exist_ok=True)

# for k in tqdm(fold_window_dict):
#     for i, window in enumerate(fold_window_dict[k]):
#         gaf = GramianAngularField(image_size=image_size, overlapping=False) # GAF는 PPA를 사용하여 시계열 축소
#         trans_data = gaf.fit_transform(window[channels].T.sort_index()).astype(np.float32)
        
#         # min-max 정규화 (gray-scale 이미지 생성을 위해)
#         for j in range(trans_data.shape[2]):
#             min_val, max_val = np.min(trans_data[:, :, j]), np.max(trans_data[:, :, j])
#             trans_data[:, :, j] = (trans_data[:, :, j] - min_val) / (max_val - min_val)
        
#         if np.isnan(trans_data).any():
#             print('NaN detected', k, {i+1})
#             continue

#         # trans_data = np.transpose(trans_data, (1, 2, 0)) # (채널, 높이, 너비) -> (높이, 너비, 채널)로 변경

#         np.save(f'../../images2/{gaf_save_dir}_fold_{"_".join(channels)}/{k}_{i+1:03d}th_window.npy', trans_data)

## MTF 저장

### epoch로 나누어진 데이터 저장

In [ ]:
## MTF 변환 (epoched data)

import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import MarkovTransitionField
from concurrent.futures import ProcessPoolExecutor
import torch
from torchvision import transforms

def process_file(file_name, trans_size=1, resized_size=(224, 224)):
    subj_name = file_name.split('/')[-1][:-4]
    data = pd.read_csv(file_name, index_col=0).drop(columns=['time', 'condition', 'A2'])
    epoch_idx = data['epoch'].unique()
    
    for idx in epoch_idx:
        window = data[data['epoch'] == idx].drop(columns=['epoch']).T
        if trans_size == 1:
            mtf = MarkovTransitionField(n_bins=5, overlapping=False)
        else:
            mtf = MarkovTransitionField(image_size=trans_size, n_bins=5, overlapping=False)

        trans_data = mtf.fit_transform(window.sort_index()).astype(np.float32)

        # min-max 정규화 (gray-scale 이미지 생성을 위해)
        for j in range(trans_data.shape[0]):
            min_val, max_val = np.min(trans_data[j, :, :]), np.max(trans_data[j, :, :])
            trans_data[j, :, :] = (trans_data[j, :, :] - min_val) / (max_val - min_val)

        resized_img = transforms.Resize(resized_size)(torch.from_numpy(trans_data)) # (c, h, w) 바로 resize
        resized_img = resized_img.numpy()
        
        if np.isnan(resized_img).any():
            print('NaN detected')
            continue

        np.save(f'../../images22/{mtf_save_dir}_fold/{subj_name}_{idx:03d}th_window.npy', resized_img)

    return f'{subj_name} 처리 완료'

def main():
    os.makedirs(f'../../images2/{mtf_save_dir}_fold', exist_ok=True)
    
    # 사용할 프로세스 수 설정 (CPU 코어 수에 따라 조정 가능)
    num_processes = 15

    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        futures = [executor.submit(process_file, file_name) for file_name in filenames]
        
        for future in tqdm(futures, total=len(filenames), desc="전체 진행률"):
            print(future.result())

if __name__ == '__main__':
    import matplotlib
    matplotlib.use('Agg')
    main()

### epochs로 나누어 지지 않은 데이터 저장

In [ ]:
# MTF 변환 (not epoched)

import os
import numpy as np
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
from pyts.image import MarkovTransitionField
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms


def process_window(k, i, window, trans_size=1, resized_size=(224, 224)):
    if trans_size == 1:
        mtf = MarkovTransitionField(n_bins=5, overlapping=False) 
    else:
        mtf = MarkovTransitionField(image_size=trans_size, n_bins=5, overlapping=False)

    trans_data = mtf.fit_transform(window.T.sort_index()).astype(np.float32)
    
    # Min-max normalization (to create a grayscale image)
    for j in range(trans_data.shape[0]):
        min_val, max_val = np.min(trans_data[j, :, :]), np.max(trans_data[j, :, :])
        trans_data[j, :, :] = (trans_data[j, :, :] - min_val) / (max_val - min_val)

    resized_img = transforms.Resize(resized_size)(torch.from_numpy(trans_data)) # (c, h, w) 바로 resize
    resized_img = resized_img.numpy()
    
    if np.isnan(resized_img).any():
        print('NaN detected', k, {i + 1})
        return

    # Save the normalized data
    np.save(f'../../images2/{mtf_save_dir}_fold/{k}_{i + 1:03d}th_window.npy', resized_img)

    return f'{k} 처리 완료'


def main():
    fold_window_dict = {}

    for k in raw_df_dict:
        fold_window_dict[k] = sliding_window(raw_df_dict[k], window_size * sfreq, overlap * sfreq)

    os.makedirs(f'../../images2/{mtf_save_dir}_fold', exist_ok=True)

    tasks = []
    num_processes = 15

    # Create a ProcessPoolExecutor to manage the pool of processes
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        for k in tqdm(fold_window_dict):
            for i, window in enumerate(fold_window_dict[k]):
                # Submit the processing function to the pool
                tasks.append(executor.submit(process_window, k, i, window, trans_size=1, resized_size=(224, 224)))
        
        # Wait for all tasks to complete
        for task in tqdm(tasks):
            task.result()


if __name__ == '__main__':
    import matplotlib
    matplotlib.use('Agg')
    main()

## Scalogram 저장

In [11]:
# Scalogram 변환 (not epoched)

import os
import numpy as np
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import pywt

def sliding_window(df, window_size, overlap):
    step = window_size - overlap
    windows = [df.iloc[i:i + window_size] for i in range(0, len(df) - window_size + 1, step)]
    return windows

def process_window(k, i, window, image_size=224, num_scales=500):
    window = window.T.sort_index().T

    # CWT 계산
    min_freq = 1
    max_freq = 256 // 2 # Nyquist 주파수
    scales = np.geomspace(min_freq, max_freq, num=num_scales)

    # 주파수를 scale로 변환 (Morlet 웨이블릿의 중심 주파수를 1로 가정)
    scales = 256 / (2 * scales * np.pi)
    
    # 다채널 이미지를 위한 초기화
    scalograms = np.zeros((image_size, image_size, window.shape[1]), dtype=np.float32)
    
    for j, channel in enumerate(window):
        # CWT 수행
        coef, _ = pywt.cwt(window[channel], scales, 'cmor1.5-1.0')
        
        # 이미지 resize
        scalogram = np.abs(coef)
        scalogram = np.flip(scalogram, axis=0)  # Flip vertically to have low frequencies at the bottom
        scalogram = resize(scalogram, (image_size, image_size), mode='constant', anti_aliasing=True)
        
        # 그레이 스케일 이미지를 위한 min-max 정규화
        scalogram = (scalogram - np.min(scalogram)) / (np.max(scalogram) - np.min(scalogram))
        
        scalograms[:, :, j] = scalogram
    
    scalograms = scalograms.transpose(2, 0, 1)
    
    if np.isnan(scalograms).any():
        print('NaN detected', k, {i + 1})
        return

    # 이미지 저장
    np.save(f'../../images2/{scalo_save_dir}_fold/{k}_{i + 1:03d}th_window.npy', scalograms)

    return f'{k} 처리 완료'


def main():
    
    fold_window_dict = {}

    for k in raw_df_dict:
        fold_window_dict[k] = sliding_window(raw_df_dict[k], int(window_size * sfreq), int(overlap * sfreq))

    os.makedirs(f'../../images2/{scalo_save_dir}_fold', exist_ok=True)

    tasks = []

    num_processes = 16

    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        for k in tqdm(fold_window_dict):
            for i, window in enumerate(fold_window_dict[k]):
                tasks.append(executor.submit(process_window, k, i, window))
        
        for task in tqdm(tasks):
            task.result()

if __name__ == '__main__':
    from skimage.transform import resize
    main()

100%|██████████| 3460/3460 [11:49<00:00,  4.88it/s]


- 진폭이 +- 100uV 인 윈도우는 폐기하는 코드

In [34]:
import os
import numpy as np
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import pywt
from skimage.transform import resize

def sliding_window(df, window_size, overlap):
    step = window_size - overlap
    windows = [df.iloc[i:i + window_size] for i in range(0, len(df) - window_size + 1, step)]
    return windows

def process_window(k, i, window, image_size=224, num_scales=500):
    window = window.T.sort_index().T

    # Check if any channel exceeds the amplitude threshold
    if np.any(np.abs(window) > 100):
        return None, f'{k}_{i + 1:03d}th_window exceeds amplitude threshold'

    # CWT 계산
    min_freq = 1
    max_freq = 256 // 2 # Nyquist 주파수
    scales = np.geomspace(min_freq, max_freq, num=num_scales)

    # 주파수를 scale로 변환 (Morlet 웨이블릿의 중심 주파수를 1로 가정)
    scales = 256 / (2 * scales * np.pi)
    
    # 다채널 이미지를 위한 초기화
    scalograms = np.zeros((image_size, image_size, window.shape[1]), dtype=np.float32)
    
    for j, channel in enumerate(window):
        # CWT 수행
        coef, _ = pywt.cwt(window[channel], scales, 'cmor1.5-1.0')
        
        # 이미지 resize
        scalogram = np.abs(coef)
        scalogram = np.flip(scalogram, axis=0)  # Flip vertically to have low frequencies at the bottom
        scalogram = resize(scalogram, (image_size, image_size), mode='constant', anti_aliasing=True)
        
        # 그레이 스케일 이미지를 위한 min-max 정규화
        scalogram = (scalogram - np.min(scalogram)) / (np.max(scalogram) - np.min(scalogram))
        
        scalograms[:, :, j] = scalogram
    
    scalograms = scalograms.transpose(2, 0, 1)
    
    if np.isnan(scalograms).any():
        return None, f'NaN detected in {k}_{i + 1:03d}th_window'

    return scalograms, f'{k}_{i + 1:03d}th_window'

def main():
    fold_window_dict = {}

    for k in raw_df_dict:
        fold_window_dict[k] = sliding_window(raw_df_dict[k], int(window_size * sfreq), int(overlap * sfreq))

    os.makedirs(f'../../images2/{scalo_save_dir}_fold', exist_ok=True)

    tasks = []

    num_processes = 16

    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        for k in tqdm(fold_window_dict):
            for i, window in enumerate(fold_window_dict[k]):
                tasks.append(executor.submit(process_window, k, i, window))
        
        for task in tqdm(tasks):
            result, message = task.result()
            if result is not None:
                np.save(f'../../images2/{scalo_save_dir}_fold/{message}.npy', result)
            else:
                print(message)  # 진폭 초과 또는 NaN 발생 메시지 출력

if __name__ == '__main__':
    main()

  0%|          | 13/3460 [00:03<09:16,  6.19it/s] 

H S1 EC_009th_window exceeds amplitude threshold
H S1 EC_010th_window exceeds amplitude threshold
H S1 EC_011th_window exceeds amplitude threshold
H S1 EC_012th_window exceeds amplitude threshold


  4%|▍         | 153/3460 [00:26<09:39,  5.70it/s]

H S11 EC_018th_window exceeds amplitude threshold
H S11 EC_019th_window exceeds amplitude threshold


  6%|▌         | 191/3460 [00:32<10:18,  5.28it/s]

...

 98%|█████████▊| 3376/3460 [08:19<00:16,  5.02it/s]

MDD S7 EC_037th_window exceeds amplitude threshold


 98%|█████████▊| 3400/3460 [08:22<00:11,  5.38it/s]

MDD S7 EC_060th_window exceeds amplitude threshold


100%|██████████| 3460/3460 [08:31<00:00,  6.76it/s]
